In [18]:
%pip install websockets
%pip install langchain
%pip install langchain_nvidia_ai_endpoints
%pip install langchain_community gradio
%pip install  faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
    --------------------------------------- 0.3/14.6 MB ? eta -:--:--
   ---------------------- ----------------- 8.4/14.6 MB 37.1 MB/s eta 0:00:01
   ------------------------- -------------- 9.4/14.6 MB 20.3 MB/s eta 0:00:01
   ------------------------- -------------- 9.4/14.6 MB 20.3 MB/s eta 0:00:01
   ------------------------------------- -- 13.6/14.6 MB 16.1 MB/s eta 0:00:01
   ---------------------------------------- 14.6/14.6 MB 15.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
import gradio as gr
from enum import Enum
# 新增知识库集成部分
from langchain_community.vectorstores import FAISS
import re


In [20]:
# 初始化FAISS向量库
def init_vector_store():
    embeddings = NVIDIAEmbeddings(
        model="NV-Embed-QA",   
        nvidia_api_key="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"
    )
    
    # 初始化示例数据（根据实际需求扩展）
    documents = [
        "北京故宫是中国明清两代的皇家宫殿",
        "上海外滩是著名的历史建筑群",
        "西安兵马俑是秦始皇的陪葬坑",
        "杭州西湖有十景包括苏堤春晓等",
        "广州塔昵称小蛮腰，高600米"
    ]
    
    return FAISS.from_texts(
        texts=documents,
        embedding=embeddings,
        metadatas=[{"source": "knowledge"}] * len(documents)  # 添加元数据
    )
init_vector_store()

In [21]:
# 状态枚举优化
class State(Enum):
    INIT = 0
    GET_PREFERENCE = 1
    SELECT_SPOT = 2 
    SELECT_ROUTE = 3
    SHOW_GUIDE = 4

# 初始化LLM
llm = ChatNVIDIA(
    model="deepseek-ai/deepseek-r1",
    temperature=0.6,
    top_p=0.7,
    max_tokens=4096,
    nvidia_api_key="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"
)


# 在process_step中添加检索逻辑
def process_step(state, user_input):
    """核心处理逻辑"""
    new_state = state.copy()
    
    try:
        # 初始状态
        if state["step"] == State.INIT:
            new_state["step"] = State.GET_PREFERENCE
            return format_response("🏖️ 请问您想要什么样的旅游呢？")
            
        # 获取旅游偏好
        elif state["step"] == State.GET_PREFERENCE:
            # 使用知识库增强用户输入
            docs = FAISS.similarity_search(user_input, k=3)
            context = "\n".join([d.page_content for d in docs])
            enhanced_input = f"背景知识:{context}\n用户需求:{user_input}"
            
            # 使用管道处理景点推荐
            result = (
                {"preference": enhanced_input} 
                | spot_prompt 
                | llm 
                | RunnableLambda(lambda x: x.content)
            ).invoke()
            
            spots = parse_options(result)
            new_state["step"] = State.SELECT_SPOT
            return format_response(result, spots)
            
        # 路线推荐步骤
        elif state["step"] == State.SELECT_SPOT:
            # 使用管道处理路线推荐
            result = (
                {"spot": user_input}
                | route_prompt
                | llm
                | RunnableLambda(lambda x: x.content)
            ).invoke()
            
            routes = parse_options(result)
            new_state["step"] = State.SELECT_ROUTE
            return format_response(result, routes)
            
        # 攻略生成步骤
        elif state["step"] == State.SELECT_ROUTE:
            # 使用管道处理攻略生成
            result = (
                {"route": user_input}
                | guide_prompt
                | llm
                | RunnableLambda(lambda x: x.content)
            ).invoke()
            
            new_state["step"] = State.INIT
            return format_response(result, markup=True)
            
    except Exception as e:
        return format_response(f"⚠️ 出错：{str(e)}，请重新输入")
    
    return format_response("未知状态")




# 使用RunnableSequence构建处理链
def create_chain(prompt_template):
    return (
        RunnablePassthrough() 
        | PromptTemplate.from_template(prompt_template)
        | llm
    )


# 优化提示模板
# 修改spot_prompt，加入历史上下文
spot_prompt = PromptTemplate(
    input_variables=["context", "preference"],
    template="""作为旅行规划专家，根据以下信息推荐3个景点：

背景知识：
{context}

用户偏好：{preference}

要求：
1. 按编号列出选项，格式：
   1. [景点名称]：[50字简介]
   2. [景点名称]：[50字简介]
   3. [景点名称]：[50字简介]
2. 每个景点用1行描述"""
)

# 修改route_prompt，加入历史上下文
route_prompt = PromptTemplate(
    input_variables=["preference", "spot"],
    template="""用户之前表达的旅游偏好：{preference}
选择的景点：{spot}

请设计3种游玩路线：
要求：
1. 按编号列出选项，格式：
   1. [路线名称]：[特色描述]
   2. [路线名称]：[特色描述]
   3. [路线名称]：[特色描述]
2. 每路线用1行描述"""
)

# 修改guide_prompt，加入历史上下文
guide_prompt = PromptTemplate(
    input_variables=["preference", "spot", "route"],
    template="""用户偏好：{preference}
选择的景点：{spot}
选择的路线：{route}

请创建详细攻略：
包含：
- 行程安排（时间表）
- 必玩项目
- 美食推荐
- 住宿建议
- 预算估算
输出格式：Markdown"""
)



def parse_options(text):
    """解析带编号的选项"""
    return re.findall(r"\d+\.\s+(.*?):", text)

def format_response(message, options=None, markup=False):
    """构建标准化响应"""
    response = {"message": message}
    if options:
        response["options"] = options
    if markup:
        response["message"] = f"```markdown\n{message}\n```"
    return response

def process_step(state, user_input):
    """核心处理逻辑"""
    new_state = state.copy()
    
    try:
        # 初始状态
        if state["step"] == State.INIT:
            new_state = {
                "step": State.GET_PREFERENCE,
                "preference": None,
                "spot": None,
                "route": None
            }
            return new_state, format_response("🏖️ 请问您想要什么样的旅游呢？")
            
        # 获取旅游偏好
        elif state["step"] == State.GET_PREFERENCE:
            new_state["preference"] = user_input
            
            # 使用知识库增强用户输入 
            # docs = vearch.similarity_search(user_input, k=3)
            # context = "\n".join([d.page_content for d in docs])
            
            # 构建正确的输入字典
            spot_chain = (
                RunnablePassthrough()  # 保持字典结构
                | spot_prompt 
                | llm 
                | RunnableLambda(lambda x: x.content)
            )
            
            result = spot_chain.invoke({
                "context": "something",
                "preference": user_input
            })
            
            spots = parse_options(result)
            new_state["step"] = State.SELECT_SPOT
            return new_state, format_response(result, spots)
            
        # 路线推荐步骤
        elif state["step"] == State.SELECT_SPOT:
            new_state["spot"] = user_input
            
            # 构建正确的输入字典
            route_chain = (
                RunnablePassthrough()  # 保持字典结构
                | route_prompt
                | llm
                | RunnableLambda(lambda x: x.content)
            )
            
            result = route_chain.invoke({
                "preference": state.get("preference", ""),
                "spot": user_input
            })
            
            routes = parse_options(result)
            new_state["step"] = State.SELECT_ROUTE
            return new_state, format_response(result, routes)
            
        # 攻略生成步骤
        elif state["step"] == State.SELECT_ROUTE:
            new_state["route"] = user_input
            
            # 构建正确的输入字典
            guide_chain = (
                RunnablePassthrough()  # 保持字典结构
                | guide_prompt
                | llm
                | RunnableLambda(lambda x: x.content)
            )
            
            result = guide_chain.invoke({
                "preference": state.get("preference", ""),
                "spot": state.get("spot", ""),
                "route": user_input
            })

            # 提取markdown内容
            md_content = result.strip('```markdown\n').strip('```')
            
            # 生成图片文件名
            output_path = f"travel_guide_{int(time.time())}.jpg"
            
            # 渲染为图片
            await render_md_to_image(md_content, output_path)
            
            new_state["step"] = State.INIT
            return new_state, format_response(
                f"攻略已生成!\n\n{result}\n\n图片已保存至: {output_path}", 
                markup=True
            )
            
    except Exception as e:
        print(f"Error details: {e}")  # 添加详细错误信息打印
        return state, format_response(f"⚠️ 出错：{str(e)}，请重新输入")
    
    return state, format_response("未知状态")

d:\project\CereBraWise\.conda\lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:197: UserWarning: Found deepseek-ai/deepseek-r1 in available_models, but type is unknown and inference may fail.
  warnings.warn(


SyntaxError: 'await' outside async function (2378145772.py, line 245)

In [9]:
# 导入所需的包
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import puppeteer
from markupsafe import Markup
import asyncio
import os

# AI设计样式生成器
async def get_ai_designed_css(md_content):
    """调用大模型生成CSS样式"""
    llm = ChatNVIDIA(
        model="deepseek-ai/deepseek-r1",
        temperature=0.7,
        nvidia_api_key="YOUR_API_KEY"
    )
    
    prompt = f"""作为专业平面设计师,请为以下Markdown内容设计海报样式:
    {md_content}
    
    要求:
    1. 使用优雅的渐变背景
    2. 合理的字体层级和间距
    3. 重要内容突出显示
    4. 适合1080x1920的手机海报尺寸
    
    请直接返回CSS代码(不要解释)。
    """
    
    response = await llm.agenerate([prompt])
    css = response.generations[0].text
    return css

# Markdown转图片渲染器
async def render_md_to_image(md_content, output_path):
    """将Markdown渲染为图片"""
    # 获取AI设计的CSS
    css = await get_ai_designed_css(md_content)
    
    # 构建完整HTML
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <style>
            {css}
            /* 基础样式 */
            body {{
                margin: 0;
                padding: 40px;
                width: 1080px;
                min-height: 1920px;
            }}
            /* Markdown基础样式 */
            h1 {{ margin-bottom: 1em; }}
            p {{ line-height: 1.6; }}
        </style>
    </head>
    <body>
        <div class="poster-container">
            {Markup(md_content)}
        </div>
    </body>
    </html>
    """
    
    # 使用Puppeteer渲染
    browser = await puppeteer.launch()
    page = await browser.newPage()
    await page.setViewport({'width': 1080, 'height': 1920})
    await page.setContent(html)
    await page.screenshot({
        'path': output_path,
        'fullPage': True,
        'type': 'jpeg',
        'quality': 90
    })
    await browser.close()


ModuleNotFoundError: No module named 'puppeteer'

In [6]:
# 添加请求间隔时间（秒）
import time


TEST_INTERVAL = 100  
# 测试代码
def test_travel_assistant():
    print("开始测试...")
    
    # 初始化状态
    state = {"step": State.INIT}
    print("初始状态:", state)
    
    # 测试初始状态
    state, response = process_step(state, "")
    print("初始响应:", response)
    print("更新后状态:", state)
    time.sleep(TEST_INTERVAL)

    # 测试获取偏好
    print("\n测试获取偏好...")
    state, response = process_step(state, "我想去一个历史文化景点，最好能体验古代宫廷文化")
    print("偏好响应:", response)
    print("更新后状态:", state)
    time.sleep(TEST_INTERVAL)

    
    # 测试选择景点
    print("\n测试选择景点...")
    state, response = process_step(state, "北京故宫")
    print("景点响应:", response)
    print("更新后状态:", state)
    time.sleep(TEST_INTERVAL)

    # 测试选择路线
    print("\n测试选择路线...")
    state, response = process_step(state, "故宫深度一日游")
    print("路线响应:", response)
    print("更新后状态:", state)

# 运行测试
test_travel_assistant()

开始测试...
初始状态: {'step': <State.INIT: 0>}
初始响应: {'message': '🏖️ 请问您想要什么样的旅游呢？'}
更新后状态: {'step': <State.GET_PREFERENCE: 1>, 'preference': None, 'spot': None, 'route': None}

测试获取偏好...
偏好响应: {'message': '<think>\n好的，用户需要推荐三个历史文化景点，特别是能体验古代宫廷文化的地方。首先，我得考虑哪些地方最符合这个主题。故宫肯定是首选，作为明清皇宫，规模大，保存完好，能全面展示宫廷生活。然后是颐和园，虽然主要是皇家园林，但里面的建筑和布局也反映了宫廷文化，尤其是昆明湖和长廊很有代表性。第三个可能需要找其他地方，比如西安的华清宫，不仅有唐代皇家温泉，还有历史事件如长恨歌的背景，能体现不同朝代的宫廷文化。另外，沈阳故宫虽然规模小一些，但作为清朝早期的宫殿，也有独特价值。不过用户可能更熟悉北京和西安的景点，所以优先考虑这两个地方。确认每个景点的简介是否在50字左右，语言要简洁明了，突出宫廷文化元素。最后检查格式是否正确，编号和描述是否在一行内。\n</think>\n\n1. [故宫博物院]：明清两代皇家宫殿，世界最大木结构建筑群，可参观太和殿、乾清宫等标志性建筑，沉浸式体验古代宫廷生活与礼仪文化。\n2. [颐和园]：清代皇家园林与行宫，包含仁寿殿宫廷区、昆明湖及长廊，完整展现皇家园林建筑艺术与帝王理政生活场景。\n3. [华清宫]：唐代皇家温泉离宫遗址，保留星辰汤等御用汤池，结合《长恨歌》实景演出，再现盛唐皇家沐浴文化与宫廷往事。'}
更新后状态: {'step': <State.SELECT_SPOT: 2>, 'preference': '我想去一个历史文化景点，最好能体验古代宫廷文化', 'spot': None, 'route': None}

测试选择景点...
景点响应: {'message': '<think>\n好的，用户之前提到想去一个历史文化景点，特别是想体验古代宫廷文化，所以他们选择了北京故宫。现在需要设计三种游玩路线，每种路线要有名称和特色描述，每行一个。\n\n首先，我需要考虑故宫的不同游览角度。可能的方向包括时间顺序、建筑特色、宫廷生活、影视

In [4]:
import gradio as gr
# multi_modal_chart_agent = gr.Interface(fn=chart_agent_gr,
#                     inputs=[gr.Image(label="Upload image", type="filepath"), 'text'],
#                     outputs=['image'],
#                     title="Multi Modal chat agent",
#                     description="Multi Modal chat agent",
#                     allow_flagging="never")

# multi_modal_chart_agent.launch(debug=True, share=False, show_api=False, server_port=5000, server_name="0.0.0.0")


# 创建对话界面
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    state = gr.State({"step": State.INIT})

    demo.load(
        fn=lambda: ([], {"step": State.INIT}),
        outputs=[chatbot, state]
    )

    with gr.Column():
        gr.Markdown("# 🌍 智能旅行助手")
        chatbot = gr.Chatbot(height=500)
        input_box = gr.Textbox(placeholder="请输入...", label="您的选择")
        choices = gr.Radio(visible=False)
        
    def handle_interaction(user_input, history, current_state):
        response = process_step(current_state, user_input)
        return (
            history + [(user_input, response["message"])],
            "",
            gr.update(visible=bool(response.get("options")), 
                    choices=response.get("options", [])),
            gr.update(visible=not bool(response.get("options"))),
            response
        )

    
    # 事件绑定修正
    input_box.submit(
        handle_interaction,
        inputs=[input_box, chatbot, state],
        outputs=[
            chatbot,  # 聊天记录
            input_box,  # 输入框内容
            choices,  # Radio组件
            input_box,  # Textbox组件（更新可见性）
            state  # 状态存储
        ]
    )
    choices.select(
        lambda x: x,
        [choices],
        [input_box]
    )


demo.launch(server_name="0.0.0.0", server_port=5000)

NameError: name 'State' is not defined

开始测试...
初始状态: {'step': <State.INIT: 0>}
初始响应: {'message': '🏖️ 请问您想要什么样的旅游呢？'}
更新后状态: {'step': <State.GET_PREFERENCE: 1>, 'preference': None, 'spot': None, 'route': None}

测试获取偏好...
偏好响应: {'message': '<think>\n好的，用户需要推荐三个历史文化景点，特别是能体验古代宫廷文化的地方。首先，我得考虑哪些地方最符合这个主题。故宫肯定是首选，作为明清皇宫，规模大，保存完好，能全面展示宫廷生活。然后是颐和园，虽然主要是皇家园林，但里面的建筑和布局也反映了宫廷文化，尤其是昆明湖和长廊很有代表性。第三个可能需要找其他地方，比如西安的华清宫，不仅有唐代皇家温泉，还有历史事件如长恨歌的背景，能体现不同朝代的宫廷文化。另外，沈阳故宫虽然规模小一些，但作为清朝早期的宫殿，也有独特价值。不过用户可能更熟悉北京和西安的景点，所以优先考虑这两个地方。确认每个景点的简介是否在50字左右，语言要简洁明了，突出宫廷文化元素。最后检查格式是否正确，编号和描述是否在一行内。\n</think>\n\n1. [故宫博物院]：明清两代皇家宫殿，世界最大木结构建筑群，可参观太和殿、乾清宫等标志性建筑，沉浸式体验古代宫廷生活与礼仪文化。\n2. [颐和园]：清代皇家园林与行宫，包含仁寿殿宫廷区、昆明湖及长廊，完整展现皇家园林建筑艺术与帝王理政生活场景。\n3. [华清宫]：唐代皇家温泉离宫遗址，保留星辰汤等御用汤池，结合《长恨歌》实景演出，再现盛唐皇家沐浴文化与宫廷往事。'}
更新后状态: {'step': <State.SELECT_SPOT: 2>, 'preference': '我想去一个历史文化景点，最好能体验古代宫廷文化', 'spot': None, 'route': None}

测试选择景点...
景点响应: {'message': '<think>\n好的，用户之前提到想去一个历史文化景点，特别是想体验古代宫廷文化，所以他们选择了北京故宫。现在需要设计三种游玩路线，每种路线要有名称和特色描述，每行一个。\n\n首先，我需要考虑故宫的不同游览角度。可能的方向包括时间顺序、建筑特色、宫廷生活、影视取景地等。用户可能希望路线各有侧重，满足不同的兴趣点。\n\n第一种路线可以是时间顺序的，按照从南到北的中轴线游览，这样能覆盖故宫的主要建筑，比如午门、三大殿、后三宫和御花园。这种路线适合第一次来故宫的游客，全面了解整体布局和历史。\n\n第二种路线可以聚焦宫廷生活，比如东西六宫和慈宁宫，这些地方展示了妃嫔的住所和太后的生活区域，加上珍宝馆的文物，能深入体验后宫文化和日常细节。\n\n第三种路线可能结合影视作品，比如《甄嬛传》的取景地，吸引喜欢清宫剧的游客。这样的路线会有代入感，让游客更有兴趣探索相关宫殿的历史背景。\n\n需要确保每个路线名称吸引人，特色描述简洁明了。还要检查路线是否合理，避免重复或遗漏重要景点。比如中轴线是经典路线，后宫生活突出细节，影视路线增加趣味性。这样三种路线各有特色，满足不同需求。\n</think>\n\n1. [中轴探秘线]：午门→太和殿→中和殿→保和殿→乾清宫→交泰殿→坤宁宫→御花园，沿明清帝王登基大典路线感受皇家威仪  \n2. [后宫生活线]：西六宫（储秀宫/翊坤宫）→东六宫（景仁宫/延禧宫）→慈宁宫→寿康宫→珍宝馆，深度体验妃嫔起居与宫廷秘史  \n3. [宫廷美学线]：文华殿陶瓷馆→武英殿书画馆→箭亭武备展→畅音阁戏台→乾隆花园，系统观赏宫廷艺术精品与建筑美学'}
更新后状态: {'step': <State.SELECT_ROUTE: 3>, 'preference': '我想去一个历史文化景点，最好能体验古代宫廷文化', 'spot': '北京故宫', 'route': None}

测试选择路线...
路线响应: {'message': '```markdown\n<think>\n好的，用户想要一个关于北京故宫的深度一日游攻略，重点在历史文化体验和古代宫廷文化。首先，我需要确定用户的需求是否明确。用户提到偏好历史文化景点，尤其是宫廷文化，所以故宫确实是最佳选择。接下来，用户希望有一个详细的时间安排、必玩项目、美食推荐、住宿建议和预算估算。输出格式是Markdown，需要结构清晰。\n\n首先，行程安排要合理，覆盖故宫的主要景点，同时考虑到时间和体力。故宫很大，一日游的话需要高效安排路线，避免遗漏重要部分。可能需要分上午、下午的时间段，每个时间段参观哪些区域。比如，上午中轴线的主要建筑，下午东西六宫和珍宝馆等。\n\n必玩项目方面，太和殿、乾清宫、珍宝馆这些肯定是重点。另外，用户可能对互动体验感兴趣，比如数字馆或者文创店，可以加入进去。还有角楼和神武门外的景山公园，俯瞰全景，这也是很多攻略会推荐的。\n\n美食推荐方面，故宫里面的餐厅比如冰窖餐厅和故宫餐厅，还有角楼咖啡，这些是必须提到的。周边的话，附近有四季民福烤鸭店，可以建议用户出去后用餐。\n\n住宿建议需要根据预算，用户可能想要不同价位的选择，比如高端酒店、中端和经济型。靠近故宫的位置会更方便，比如王府井、东华门附近的酒店。\n\n预算估算要分门票、餐饮、交通、住宿和其他费用。门票价格要准确，淡旺季是否有区别？目前故宫的门票是60元，珍宝馆和钟表馆各10元。餐饮的话，故宫内的餐厅人均大约50-100，周边餐厅可能更高。交通主要是地铁和打车，住宿价格区间要合理。\n\n需要注意的细节：开放时间，周一闭馆，提前预约，穿舒适的鞋子，带身份证等。这些提醒对用户来说很重要，避免他们到了之后遇到问题。\n\n可能用户没有明确提到的是对故宫历史的兴趣程度，是否需要讲解服务或者导览器租赁，这部分可以加入必玩项目中，建议租用导览器或请导游，提升体验。另外，拍照建议，比如哪些地点适合拍照，但用户可能也关心人流情况，如何避开高峰。\n\n还要考虑季节因素，比如夏季需要防晒，冬季保暖，但用户没有特别说明，所以可能保持通用建议。时间安排是否合理，比如早上8:30入场，中午用餐时间安排在哪里
，下午的路线是否顺畅，不会走回头路。\n\n最后，预算部分要总计，给出一个范围，让用户有大致的概念。可能需要检查每个部分的价格是否准确，比如门票、餐饮、住宿的价格是否有变动，需要最新数据。比如故宫现在的门票价格是60，珍宝馆10，钟表馆10，加起来80。餐饮部分，冰窖餐厅的人均可能50-100，角楼咖啡30-50，晚餐烤鸭店人均150左右。住宿的话，高端酒店如王府井附近可能1000以上，中端500-1000，经济型200-500。\n\n总结下来，需要结构清晰，信息准确，涵盖用户需求的所有方面，同时给出实用建议，如提前预约、路线优化等。确保每个部分都详细但不过于冗长，适合一日游的节奏。\n</think>\n\n```markdown\n# 北京故宫深度一日游攻略\n\n## 🕒 行程安排（时间表）\n| 时间段       | 活动内容                                                                 |\n|--------------|--------------------------------------------------------------------------|\n| **08:30-09:00** | 抵达故宫午门（建议提前预约门票，刷身份证入场）                          |\n| **09:00-11:30** | **中轴线深度游**<br>- 太和殿（金銮殿）→ 中和殿 → 保和殿 → 乾清宫 → 交泰殿 → 坤宁宫 → 御花园 |\n| **11:30-13:00** | **午餐+文化体验**<br>- 冰窖餐厅（宫廷风格餐厅）用餐<br>- 参观故宫文创店购买特色手信 |\n| **13:00-15:30** | **东西六宫探索**<br>- 西六宫（储秀宫、翊坤宫）→ 东六宫（延禧宫、景仁宫）→ 珍宝馆（必看：金瓯永固杯、点翠凤冠）→ 钟表馆 |\n| **15:30-16:30** | **角楼与城墙**<br>- 登东华门城墙俯瞰故宫全景 → 角楼拍照打卡               |\n| **16:30-17:00** | 从神武门离场，步行至景山公园万春亭俯瞰故宫全景（可选）                   |\n\n---\n\n## 🏛️ 必玩项目\n1. **太和殿广场**  \n   - 感受“天子登基”的震撼场景，观察殿前青铜鹤与日晷。\n2. **珍宝馆（宁寿宫区）**  \n   - 必看：乾隆御用金编钟、孝端皇后九龙九凤冠。\n3. **延禧宫西洋风水晶宫**  \n   - 晚清未完工的欧式建筑遗址，网红拍照点。\n4. **神武门数字故宫体验馆**  \n   - 通过VR技术沉浸式体验宫廷生活（需提前预约）。\n\n---\n\n## 🍜 美食推荐\n| 类型       | 推荐地点                          | 特色菜品                                      | 人均消费 |\n|------------|-----------------------------------|-----------------------------------------------|----------|\n| **宫廷菜**   | 冰窖餐厅（慈宁宫旁）            | 慈禧同款“菊花锅子”、故宫烤鸭                  | ¥80-120  |\n| **轻食咖啡** | 角楼咖啡（神武门外）            | 千里江山卷、康熙最爱巧克力                     | ¥30-50   |\n| **京味老字号**| 四季民福（东华门店）            | 老北京炸酱面、宫廷杏仁豆腐（出宫后步行10分钟） | ¥100-150 |\n\n---\n\n## 🏨 住宿建议\n| 类型       | 推荐酒店                          | 亮点                                      | 参考价格 |\n|------------|-----------------------------------|-------------------------------------------|----------|\n| **高端体验** | 北京王府井文华东方酒店          | 露台直面东华门，提供故宫主题下午茶        | ¥3000+/晚|\n| **中端优选** | 北京金茂万丽酒店                | 部分客房可俯瞰故宫建筑群                  | ¥800-1500/晚|\n| **性价比之选**| 时光漫步怀旧酒店（东四店）      | 胡同里的四合院改造，步行15分钟到故宫       | ¥400-600/晚|\n\n---\n\n## 💰 预算估算（单人）\n| 项目       | 费用明细                          | 金额       |\n|------------|-----------------------------------|------------|\n| **门票**     | 故宫大门票+珍宝馆+钟表馆          | ¥80        |\n| **餐饮**     | 午餐+下午茶+晚餐                  | ¥150-250   |\n| **交通**     | 地铁/打车（市内往返）             | ¥20-50     |\n| **住宿**     | 经济型/中端/高端                  | ¥400-3000  |\n| **其他**     | 讲解器租赁（¥20）、文创纪念品      | ¥50-200    |\n| **总计**     |                                   | **¥700-3580** |\n\n---\n\n## 📌 注意事项\n1. 周一闭馆！需提前7天在【故宫博物院】小程序抢票（晚8点放票）\n2. 穿舒适运动鞋（日均步行2万步+）\n3. 携带身份证+充电宝（宫内无租借点）\n4. 冬季推荐红墙白雪拍照，夏季建议早晨优先游览中轴线避暑\n```\n```'}
更新后状态: {'step': <State.INIT: 0>, 'preference': '我想去一个历史文化景点，最好能体验古代宫廷文化', 'spot': '北京故宫', 'route': '故宫深度一日游'}


好的，用户想要一个关于北京故宫的深度一日游攻略，重点在历史文化体验和古代宫廷文化。首先，我需要确定用户的需求是否明确。用户提到偏好历史文化景点，尤其是宫廷文化，所以故宫确实是最佳选择。接下来，用户希望有一个详细的时间安排、必玩项目、美食推荐、住宿建议和预算估算。输出格式是Markdown，需要结构清晰。\n\n首先，行程安排要合理，覆盖故宫的主要景点，同时考虑到时间和体力。故宫很大，一日游的话需要高效安排路线，避免遗漏重要部分。可能需要分上午、下午的时间段，每个时间段参观哪些区域。比如，上午中轴线的主要建筑，下午东西六宫和珍宝馆等。\n\n必玩项目方面，太和殿、乾清宫、珍宝馆这些肯定是重点。另外，用户可能对互动体验感兴趣，比如数字馆或者文创店，可以加入进去。还有角楼和神武门外的景山公园，俯瞰全景，这也是很多攻略会推荐的。\n\n美食推荐方面，故宫里面的餐厅比如冰窖餐厅和故宫餐厅，还有角楼咖啡，这些是必须提到的。周边的话，附近有四季民福烤鸭店，可以建议用户出去后用餐。\n\n住宿建议需要根据预算，用户可能想要不同价位的选择，比如高端酒店、中端和经济型。靠近故宫的位置会更方便，比如王府井、东华门附近的酒店。\n\n预算估算要分门票、餐饮、交通、住宿和其他费用。门票价格要准确，淡旺季是否有区别？目前故宫的门票是60元，珍宝馆和钟表馆各10元。餐饮的话，故宫内的餐厅人均大约50-100，周边餐厅可能更高。交通主要是地铁和打车，住宿价格区间要合理。\n\n需要注意的细节：开放时间，周一闭馆，提前预约，穿舒适的鞋子，带身份证等。这些提醒对用户来说很重要，避免他们到了之后遇到问题。\n\n可能用户没有明确提到的是对故宫历史的兴趣程度，是否需要讲解服务或者导览器租赁，这部分可以加入必玩项目中，建议租用导览器或请导游，提升体验。另外，拍照建议，比如哪些地点适合拍照，但用户可能也关心人流情况，如何避开高峰。\n\n还要考虑季节因素，比如夏季需要防晒，冬季保暖，但用户没有特别说明，所以可能保持通用建议。时间安排是否合理，比如早上8:30入场，中午用餐时间安排在哪里
，下午的路线是否顺畅，不会走回头路。\n\n最后，预算部分要总计，给出一个范围，让用户有大致的概念。可能需要检查每个部分的价格是否准确，比如门票、餐饮、住宿的价格是否有变动，需要最新数据。比如故宫现在的门票价格是60，珍宝馆10，钟表馆10，加起来80。餐饮部分，冰窖餐厅的人均可能50-100，角楼咖啡30-50，晚餐烤鸭店人均150左右。住宿的话，高端酒店如王府井附近可能1000以上，中端500-1000，经济型200-500。\n\n总结下来，需要结构清晰，信息准确，涵盖用户需求的所有方面，同时给出实用建议，如提前预约、路线优化等。确保每个部分都详细但不过于冗长，适合一日游的节奏。\n
```markdown\n# 北京故宫深度一日游攻略\n\n## 🕒 行程安排（时间表）\n| 时间段       | 活动内容                                                                 |\n|--------------|--------------------------------------------------------------------------|\n| **08:30-09:00** | 抵达故宫午门（建议提前预约门票，刷身份证入场）                          |\n| **09:00-11:30** | **中轴线深度游**<br>- 太和殿（金銮殿）→ 中和殿 → 保和殿 → 乾清宫 → 交泰殿 → 坤宁宫 → 御花园 |\n| **11:30-13:00** | **午餐+文化体验**<br>- 冰窖餐厅（宫廷风格餐厅）用餐<br>- 参观故宫文创店购买特色手信 |\n| **13:00-15:30** | **东西六宫探索**<br>- 西六宫（储秀宫、翊坤宫）→ 东六宫（延禧宫、景仁宫）→ 珍宝馆（必看：金瓯永固杯、点翠凤冠）→ 钟表馆 |\n| **15:30-16:30** | **角楼与城墙**<br>- 登东华门城墙俯瞰故宫全景 → 角楼拍照打卡               |\n| **16:30-17:00** | 从神武门离场，步行至景山公园万春亭俯瞰故宫全景（可选）                   |\n\n---\n\n## 🏛️ 必玩项目\n1. **太和殿广场**  \n   - 感受“天子登基”的震撼场景，观察殿前青铜鹤与日晷。\n2. **珍宝馆（宁寿宫区）**  \n   - 必看：乾隆御用金编钟、孝端皇后九龙九凤冠。\n3. **延禧宫西洋风水晶宫**  \n   - 晚清未完工的欧式建筑遗址，网红拍照点。\n4. **神武门数字故宫体验馆**  \n   - 通过VR技术沉浸式体验宫廷生活（需提前预约）。\n\n---\n\n## 🍜 美食推荐\n| 类型       | 推荐地点                          | 特色菜品                                      | 人均消费 |\n|------------|-----------------------------------|-----------------------------------------------|----------|\n| **宫廷菜**   | 冰窖餐厅（慈宁宫旁）            | 慈禧同款“菊花锅子”、故宫烤鸭                  | ¥80-120  |\n| **轻食咖啡** | 角楼咖啡（神武门外）            | 千里江山卷、康熙最爱巧克力                     | ¥30-50   |\n| **京味老字号**| 四季民福（东华门店）            | 老北京炸酱面、宫廷杏仁豆腐（出宫后步行10分钟） | ¥100-150 |\n\n---\n\n## 🏨 住宿建议\n| 类型       | 推荐酒店                          | 亮点                                      | 参考价格 |\n|------------|-----------------------------------|-------------------------------------------|----------|\n| **高端体验** | 北京王府井文华东方酒店          | 露台直面东华门，提供故宫主题下午茶        | ¥3000+/晚|\n| **中端优选** | 北京金茂万丽酒店                | 部分客房可俯瞰故宫建筑群                  | ¥800-1500/晚|\n| **性价比之选**| 时光漫步怀旧酒店（东四店）      | 胡同里的四合院改造，步行15分钟到故宫       | ¥400-600/晚|\n\n---\n\n## 💰 预算估算（单人）\n| 项目       | 费用明细                          | 金额       |\n|------------|-----------------------------------|------------|\n| **门票**     | 故宫大门票+珍宝馆+钟表馆          | ¥80        |\n| **餐饮**     | 午餐+下午茶+晚餐                  | ¥150-250   |\n| **交通**     | 地铁/打车（市内往返）             | ¥20-50     |\n| **住宿**     | 经济型/中端/高端                  | ¥400-3000  |\n| **其他**     | 讲解器租赁（¥20）、文创纪念品      | ¥50-200    |\n| **总计**     |                                   | **¥700-3580** |\n\n---\n\n## 📌 注意事项\n1. 周一闭馆！需提前7天在【故宫博物院】小程序抢票（晚8点放票）\n2. 穿舒适运动鞋（日均步行2万步+）\n3. 携带身份证+充电宝（宫内无租借点）\n4. 冬季推荐红墙白雪拍照，夏季建议早晨优先游览中轴线避暑\n```\n```